# Scrapping du Contexte d’utilisation

Afin de détailler précisément les informations que nous avons, nous souhaitons connaitre le contexte d’utilisation de l’hyperlien vers le site du Parc National.

S’agit-il d’une citation dans un texte ? Est-il dans une liste ? La liste est non exhaustive.

## Définition des librairies

Pour ce dernier script, nous aurons besoin d’un total de 6 librairies :


In [ ]:
#************************[ Librairies Utilisées ]************************************************************************************

import pandas as pd #Gestion des Dataframes & Import/ Export Csv

import requests
from bs4 import BeautifulSoup #Librairie scraping
import re # Permet la recherche partielle sur les pages
import time #pause du script pour évite blacklisting de l'adresse IP

from tqdm import tqdm #Affichage d'une barre de progression du script
tqdm.pandas()  #permet l'utilisation de la fonction pd.progress_apply pour voir la progression du script

* ``Pandas `` nous permettra de gérer le volume important de données présentes dans les trois fichiers CSV que nous avons comme source.
* ``Requests``, nous permettra de récupérer le contenu des pages web recensées.
* ``Bs4 ``, est la librairie qui nous permettra de traiter le contexte sur les pages récupérées.
* ``re``, qui nous permettra d’effectuer une recherche partielle sur la page
* ``time ``, nous permettra d’éviter un blacklisting potentiel afin de pouvoir continuer à accéder aux sites comme bandol-blog.com qui posséde un grand nombre de pages.
* ``tdqm ``, qui nous permettra d’activer la fonction progress_apply avec la librairie Pandas. Cette dernière nous permettra d’avoir un visuel de la vitesse de traitement des données.


## Les fonctions de ce script


Ce script fonctionnera grâce à la fonction ``Scrapper``.
Cette dernière récupérera deux informations : 
- `Referring` (l’adresse de la page de l’acteur)
- `Cible` (le nom de domaine utilisé sur la page de l’acteur)

À partir de ces deux informations, la fonction grâce à la librairie requests va enregistrer le contenu de la page Referring.
La librairie ``Bs4 `` sera ensuite utilisée, et plus précisément sa fonction `BeautifulSoup` qui va lire le contenu et le traiter, ce qui va nous permettre de rechercher dans la page les balises HTML "a" (qui contiennent les hyperliens).

Puis grâce à la librairie ``re`` nous allons filtrer de manière à ne récupérer que la balise a qui contiendra en référence une adresse comportant le Nom de Domaine.

Ce contenu sera stocké dans la variable `linkP` qui sera renvoyée comme résultat.

Entre chaque lancement de la fonction, nous aurons un `time. sleep` qui évitera un potentiel blacklisting de notre script.


In [ ]:
#************************[ Fonctions Utilisées ]************************************************************************************

def Scrapper(Referring,Cible):
    req =''
    soup =''
    link =''
    linkP=''
    if Referring =='':
        pass
    else:
        req = requests.get(Referring) # La variable req récupére l'adresse présente dans 'Referring Page URL'
        soup = BeautifulSoup(req.text, 'lxml') #BeautifulSoup récupére la totalité de la page signalée au dessus
        for link in soup.find_all('a', href=re.compile(Cible)): #la fonction recherche le lien renvoyant vers la page de PortCros sur la page stockée
            linkP = link.parent #On récupére le contexte du lien présent sur la page
        time.sleep(2) #On marque une pause pour éviter le blacklisting potentiel de l'adresse IP sur le site.
    return linkP #On retourne le resultat dans DfVar['Scrapped']

## Prétraitement des données

Cette fonction se basera sur le fichier `appended.csv` que nous avons créé lors de l’exécution de notre première fonction.

Cependant, comme pour le géoréférencement, nous avons choisi de limiter notre recherche de contexte aux acteurs varois.
De ce fait, nous allons également charger `DNS_Adresses_Filtrees.csv` que nous avons créé lors de la définition de notre géoréférencement.

Grâce à ces deux fichiers, nous allons effectuer un "Merge" dans la nouvelle variable `DfVar`, qui sera paramétrée de manière à ne récupérer que le contenu commun aux deux fichiers.


In [ ]:
#************************[ Traitement Initial ]************************************************************************************
Df = pd.read_csv('appended.csv', encoding='utf-16', sep='\t') #Ouverture du csv contenant la totalité des adresses recensées
DfFilter = pd.read_csv('DNS_Adresses_Filtrees.csv', encoding='utf-16', sep='\t') #Ouverture du csv contenant les adresses varoises

DfVar = pd.merge(Df, DfFilter, on='DNS Source', how='inner') #On fusionne les deux csv précédents en ne gardant que les adresses ayant un dns en commun

print(DfVar.info(verbose=False))

Nous passerons donc d’un traitement de 286 696 références à un traitement de 41 119 références.
Soit 1/7 ème du fichier initial.

## Initialisation du script

Nous allons donc initialiser le script à partir de la variable `DfVar`.
Nous déterminons une nouvelle colonne dans les données qui sera nommée `Scrapped` et qui contiendra le retour de la fonction ``Scrapper``.
Nous utilisons la fonction Pandas progress_apply dans laquelle nous allons déterminer les correspondances entre la variable et la fonction `DfVar`.
La Colonne `Referring Page URL` correspondra à la variable referring de Scrapper et DNS Cible à la partie Cible de Scrapper.

De plus grâce à `progress_apply` nous pourrons voir l’évolution de la progression du traitement des données ainsi qu’une estimation du temps restant avant la fin de ce dernier.


In [ ]:
#************************[  ]************************************************************************************

DfVar['Scrapped'] =DfVar.progress_apply(lambda x : Scrapper(x['Referring Page URL'],x ['DNS Cible']),axis=1) #On lance la fonction Scrapper en se basant sur les Colonnes énumérées : le résultat arrive dans "Scrapped"

Pour finir, nous restituerons les données dans un dernier fichier Csv nommé `PortCrosScrapped.csv`.

In [ ]:
#************************[ Restitution des données ]************************************************************************************

DfVar.to_csv('PortCrosScrapped.csv', encoding="utf-16", sep='\t') #Fichier de sortie comportant l'ensemble des données récupérées entre les différentes étapes pour les adresses varoises.

In [1]:
%reset -f

SyntaxError: invalid syntax (<ipython-input-1-0ea39e66e148>, line 3)